In [2]:
from tqdm import tqdm
import time
import json
from enum import Enum
from gurobipy import Model, GRB, quicksum, max_
import pandas as pd
from random import randint
import plotly.graph_objs as go
import numpy as np
from numpy import array


In [3]:
class DATASET(Enum):
    TOY = "toy"
    MEDIUM = "medium"
    LARGE = "large"

# Fonctions pour préparer le modèle


In [4]:
def load_data(name):
    """name must be an instance of DATASET like DATASET.TOY for example"""
    if not isinstance(name, DATASET):
        raise TypeError("direction must be an instance of DATASET Enum")
    with open(f"../data/{name.value}_instance.json", "r") as f:
        data = json.load(f)
    return data


def get_dims(data):
    return (
        len(data["staff"]),
        data["horizon"],
        len(data["qualifications"]),
        len(data["jobs"]),
    )

def get_qualification_index(list_qualifications, qualification): # qualification is "A", "B", "C" ...
    return list_qualifications.index(qualification)

In [5]:
def init_model():
    m = Model("Project modelling")
    return m

In [6]:
def create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs):
    X = model.addVars(n_staff, horizon, n_qualifs, n_jobs, vtype=GRB.BINARY, name="assignements")
    # profit 
    profit = model.addVar(lb =0, vtype = GRB.INTEGER, name = "profit")
    J = model.addVars(n_jobs, vtype=GRB.BINARY, name="completion")
    E_D = model.addVars(n_jobs, lb=0, ub=horizon, vtype=GRB.INTEGER, name="end_dates")
    L = model.addVars(n_jobs, lb=0, ub=horizon + 1, vtype=GRB.INTEGER, name="n_days_late")

    # for max_days
    S_D = model.addVars(n_jobs, lb=0, ub=horizon, vtype=GRB.INTEGER, name="start_dates")
    spans = model.addVars(n_jobs, lb=0, ub=horizon, vtype=GRB.INTEGER, name="spans")
    max_days = model.addVar(lb=0, ub=horizon, vtype=GRB.INTEGER, name="max_jobs")

    # for max_jobs 
    n_jobs_per_person = model.addVars(n_staff, lb=0, ub=n_jobs, vtype=GRB.INTEGER, name="n_jobs_per_person")
    jobs_worked_on_by_person = model.addVars(n_staff, n_jobs, vtype=GRB.BINARY, name="jobs_worked_on_by_person")
    n_worked_days_per_job_and_person = model.addVars(n_staff, n_jobs,lb=0, ub=horizon, vtype=GRB.INTEGER, name="n_worked_days_per_job_and_person")
    max_jobs = model.addVar(lb=0, ub=n_jobs, vtype=GRB.INTEGER, name="max_jobs")
    
    return model, X, J, S_D, E_D, L, n_jobs_per_person, jobs_worked_on_by_person, n_worked_days_per_job_and_person, max_jobs, max_days, spans, profit


In [7]:
def add_constraints_start_dates(model, X, S_D, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        S_D[l] <= X[i, j, k, l] * j + (1 - X[i, j, k, l]) * horizon
        for i in range(n_staff)
        for j in range(horizon) 
        for k in range(n_qualifs)
        for l in range(n_jobs)
    )

In [8]:
def add_constraints_end_dates(model, X, E_D, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        X[i, j, k, l] * j <= E_D[l] 
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
    )

In [9]:
def add_constraints_lateness(model, E_D, L, jobs, n_staff, horizon, n_qualifs, n_jobs):
    model.addConstrs(
        E_D[l] +1 - jobs[l]["due_date"] <= L[l] for l in range(n_jobs)
    )

In [10]:
def add_constraints_worked_days_below_required_days(model, X, jobs, qualifications, n_staff, horizon, n_jobs):
    model.addConstrs(quicksum(X[i,j,get_qualification_index(qualifications, k),l] for i in range(n_staff) for j in range(horizon)) <= jobs[l]["working_days_per_qualification"][k] 
                     for l in range(n_jobs) 
                     for k in jobs[l]["working_days_per_qualification"].keys())
    

In [11]:
def add_constraints_worked_days_above_required_days(model, X, J, jobs, qualifications, n_staff, horizon, n_jobs):
    model.addConstrs(quicksum(X[i,j,get_qualification_index(qualifications, k),l] for i in range(n_staff) for j in range(horizon)) >= J[l]* jobs[l]["working_days_per_qualification"][k] 
                     for l in range(n_jobs) 
                     for k in jobs[l]["working_days_per_qualification"].keys())


In [12]:
def add_constraints_employees_working_only_one_day(model, X, J, data, n_staff, n_jobs, horizon, n_qualifs):
    model.addConstrs(quicksum(X[i,j,k,l] for l in range(n_jobs) for k in range (n_qualifs)) <= 1 
                     for i in range(n_staff) 
                     for j in range(horizon))

In [13]:
def in_qualification(data, i, k):
    return data["qualifications"][k] in data["staff"][i]["qualifications"]


def add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if not in_qualification(data, i, k)
    )


def in_vacation(i, j, data):
    data = {l: data["staff"][l] for l in range(len(data["staff"]))}
    data = data[i]["vacations"]
    return j in data


def add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data):
    model.addConstrs(
        X[i, j, k, l] == 0
        for i in range(n_staff)
        for j in range(horizon)
        for k in range(n_qualifs)
        for l in range(n_jobs)
        if in_vacation(i, j, data)
    )

In [14]:
def add_constraint_profit(model, J, L, profit, jobs):
    model.addConstr(profit == quicksum( (J[index_job] * job["gain"] - job["daily_penalty"] * L[index_job]) for index_job, job in enumerate(jobs)))

In [15]:
def add_constraints_n_worked_days_per_jobs_person(model, X, n_worked_days_per_job_and_person, n_staff, n_jobs, horizon, n_qualifs):
    model.addConstrs( n_worked_days_per_job_and_person[i, l] == quicksum( X[i,j,k,l] for j in range(horizon) for k in range(n_qualifs)) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    )

def add_constraints_jobs_worked_on_by_person(model, jobs_worked_on_by_person, n_worked_days_per_job_and_person, n_staff, n_jobs):
    model.addConstrs((jobs_worked_on_by_person[i, l] == 0) >> (n_worked_days_per_job_and_person[i,l] == 0) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    ) 
    model.addConstrs((jobs_worked_on_by_person[i, l] == 1) >> (n_worked_days_per_job_and_person[i,l] >= 1) 
        for i in range(n_staff) 
        for l in range(n_jobs)
    )

def add_constraints_n_jobs_per_person(model, n_jobs_per_person, jobs_worked_on_by_person, n_staff, n_jobs):
    model.addConstrs(n_jobs_per_person[i] == quicksum( jobs_worked_on_by_person[i, l] for l in range(n_jobs) ) for i in range(n_staff))

def add_constraint_max_jobs(model, max_jobs, n_jobs_per_person, jobs_worked_on_by_person, n_staff):
    model.addConstr(max_jobs == max_([n_jobs_per_person[i] for i in range(n_staff)]))

In [16]:
def add_constraint_spans(model, spans, E_D, S_D, n_jobs):
    model.addConstrs((spans[l] == (E_D[l] - S_D[l])+1) for l in range(n_jobs))

def add_constraint_max_days(model, max_days, spans, n_jobs):
    for l in range(n_jobs):
        model.addConstr(max_days >= spans[l]) 

In [17]:
def add_profit_as_first_objective(model, profit):
    model.setObjective(
        profit,
        GRB.MAXIMIZE
    )
    return model, profit


In [18]:
def add_minimax_jobs_as_second_objective(model, max_jobs):
    model.setObjective(
        max_jobs,
        GRB.MINIMIZE
    )
    return model

In [19]:
def add_minimax_days_spent_as_third_objective(model, max_days):
    model.setObjective(
        max_days,
        GRB.MINIMIZE
    )
    return model

In [20]:
def add_mono_objective(model, profit, max_days, max_jobs):
    model.setObjective(
        10 * profit - max_days - max_jobs,
        GRB.MAXIMIZE
    )
    return model, profit, max_days, max_jobs

# Surface

In [21]:
def prepare_model(data):
    n_staff, horizon, n_qualifs, n_jobs = get_dims(data)

    # Instanciation du modèle
    model = init_model()
    
    # Création des variables : binaires dans X et J, entières de 0 à horizon + 3
    model, X, J, S_D, E_D, L, n_jobs_per_person, jobs_worked_on_by_person, n_worked_days_per_job_and_person, max_jobs, max_days, spans, profit = create_decision_variables(model, n_staff, horizon, n_qualifs, n_jobs)
    
    # maj du modèle
    model.update()
    
    # Ajout des constraintes
    add_constraints_employees_working_only_one_day(model, X ,J,data,n_staff,n_jobs,horizon, n_qualifs)
    add_qualification_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)
    add_vacation_constraints(model, n_staff, horizon, n_qualifs, n_jobs, X, data)
    
    add_constraints_start_dates(model, X, S_D, n_staff, horizon, n_qualifs, n_jobs)
    add_constraints_end_dates(model, X, E_D, n_staff, horizon, n_qualifs, n_jobs)
    add_constraints_lateness(model, E_D, L, data["jobs"], n_staff, horizon, n_qualifs, n_jobs)
    
    add_constraints_worked_days_below_required_days(model, X, data["jobs"], data["qualifications"], n_staff, horizon, n_jobs)
    add_constraints_worked_days_above_required_days(model, X , J, data["jobs"], data["qualifications"], n_staff, horizon, n_jobs)
    
    add_constraints_n_worked_days_per_jobs_person(model, X, n_worked_days_per_job_and_person, n_staff, n_jobs, horizon, n_qualifs)
    add_constraints_jobs_worked_on_by_person(model, jobs_worked_on_by_person, n_worked_days_per_job_and_person, n_staff, n_jobs)
    add_constraints_n_jobs_per_person(model, n_jobs_per_person, jobs_worked_on_by_person, n_staff, n_jobs)
    add_constraint_spans(model, spans, E_D, S_D, n_jobs)
    
    add_constraint_profit(model, J, L, profit, data["jobs"])
    add_constraint_max_jobs(model, max_jobs, n_jobs_per_person, jobs_worked_on_by_person, n_staff)
    add_constraint_max_days(model, max_days, spans, n_jobs)
    
    
    # Fonction Objectif
    model, profit = add_profit_as_first_objective(model, profit)
    
    # maj du modèle
    model.update()
    
    # Paramétrage (mode mute)
    model.params.outputflag = 0

    return model, profit, max_days, max_jobs

In [22]:
def run_one_step(data, n_jobs, points, step):
    for j in tqdm(range(n_jobs, -1, -1)):
        model, profit, max_days, max_jobs = prepare_model(data)
        model.addConstr(max_jobs <= j)
        model.addConstr(max_days <= step)
        model.update()
        model.optimize()
        point = np.array([round(step), round(j), round(profit.X)])
        points.append(point)
    return points

In [23]:
def main(data):
    n_staff, horizon, n_qualifs, n_jobs = get_dims(data)
    points = []
    for i in tqdm(range(horizon, -1,-1)):
        for j in range(n_jobs, -1, -1):
            print(j, " / ", n_jobs)
            model, profit, max_days, max_jobs = prepare_model(data)
            model.addConstr(max_jobs <= j)
            model.addConstr(max_days <= i)
            model.update()
            model.optimize()
            point = np.array([round(i), round(j), round(profit.X)])
            points.append(point)
    return points

# Computing points

Pour calculer la surface par étape : une cellule = une contrainte max_days <= t , une boucle for contrainte max_jobs <= j

Le tout met très longtemps à tourner

In [24]:
dataset = DATASET.MEDIUM
data = load_data(dataset)
n_staff, horizon, n_qualifs, n_jobs = get_dims(data)
points = []   
    

In [ ]:
points = run_one_step(data, n_jobs, points, 22)


In [ ]:
points = run_one_step(data, n_jobs, points, 21)


In [ ]:
points = run_one_step(data, n_jobs, points, 20)


In [ ]:
points = run_one_step(data, n_jobs, points, 19)


In [ ]:
points = run_one_step(data, n_jobs, points, 18)


In [ ]:
points = run_one_step(data, n_jobs, points, 17)


In [ ]:
points = run_one_step(data, n_jobs, points, 16)


In [ ]:
points = run_one_step(data, n_jobs, points, 15)


In [ ]:
points = run_one_step(data, n_jobs, points, 14)


In [ ]:
points = run_one_step(data, n_jobs, points, 13)


In [ ]:
points = run_one_step(data, n_jobs, points, 12)


In [ ]:
points = run_one_step(data, n_jobs, points, 11)

In [ ]:
points = run_one_step(data, n_jobs, points, 10)


In [ ]:
points = run_one_step(data, n_jobs, points, 9)


In [ ]:
points = run_one_step(data, n_jobs, points, 8)


In [ ]:
points = run_one_step(data, n_jobs, points, 7)


In [ ]:
points = run_one_step(data, n_jobs, points, 6)


In [ ]:
points = run_one_step(data, n_jobs, points, 5)


In [ ]:
points = run_one_step(data, n_jobs, points, 4)


In [ ]:
points = run_one_step(data, n_jobs, points, 3)


In [ ]:
points = run_one_step(data, n_jobs, points, 2)


In [ ]:
points = run_one_step(data, n_jobs, points, 1)


In [ ]:
points = run_one_step(data, n_jobs, points, 0)


In [ ]:
points = np.array(points)

# Compute with main

Ne pas Run !! Met très longtemps !

In [ ]:
dataset = DATASET.MEDIUM
data = load_data(dataset)
n_staff, horizon, n_qualifs, n_jobs = get_dims(data)
points = main(data)

# Visualisation

Grâce au calcul de surface étape par étape, j'ai réussi à aller jusqu'à max_days <= 8, et à récupérer les résultats, que j'ai mis ici.

In [25]:
points = [array([ 22,  15, 413]), array([ 22,  14, 413]), array([ 22,  13, 413]), array([ 22,  12, 413]), array([ 22,  11, 413]), array([ 22,  10, 413]), array([ 22,   9, 413]), array([ 22,   8, 413]), array([ 22,   7, 413]), array([ 22,   6, 413]), array([ 22,   5, 413]), array([ 22,   4, 405]), array([ 22,   3, 384]), array([ 22,   2, 326]), array([ 22,   1, 194]), array([22,  0,  0]), array([ 21,  15, 413]), array([ 21,  14, 413]), array([ 21,  13, 413]), array([ 21,  12, 413]), array([ 21,  11, 413]), array([ 21,  10, 413]), array([ 21,   9, 413]), array([ 21,   8, 413]), array([ 21,   7, 413]), array([ 21,   6, 413]), array([ 21,   5, 413]), array([ 21,   4, 405]), array([ 21,   3, 384]), array([ 21,   2, 326]), array([ 21,   1, 194]), array([21,  0,  0]), array([ 20,  15, 413]), array([ 20,  14, 413]), array([ 20,  13, 413]), array([ 20,  12, 413]), array([ 20,  11, 413]), array([ 20,  10, 413]), array([ 20,   9, 413]), array([ 20,   8, 413]), array([ 20,   7, 413]), array([ 20,   6, 413]), array([ 20,   5, 413]), array([ 20,   4, 405]), array([ 20,   3, 384]), array([ 20,   2, 326]), array([ 20,   1, 194]), array([20,  0,  0]), array([ 19,  15, 413]), array([ 19,  14, 413]), array([ 19,  13, 413]), array([ 19,  12, 413]), array([ 19,  11, 413]), array([ 19,  10, 413]), array([ 19,   9, 413]), array([ 19,   8, 413]), array([ 19,   7, 413]), array([ 19,   6, 413]), array([ 19,   5, 413]), array([ 19,   4, 405]), array([ 19,   3, 384]), array([ 19,   2, 326]), array([ 19,   1, 194]), array([19,  0,  0]), array([ 18,  15, 413]), array([ 18,  14, 413]), array([ 18,  13, 413]), array([ 18,  12, 413]), array([ 18,  11, 413]), array([ 18,  10, 413]), array([ 18,   9, 413]), array([ 18,   8, 413]), array([ 18,   7, 413]), array([ 18,   6, 413]), array([ 18,   5, 413]), array([ 18,   4, 405]), array([ 18,   3, 384]), array([ 18,   2, 326]), array([ 18,   1, 194]), array([18,  0,  0]), array([ 17,  15, 413]), array([ 17,  14, 413]), array([ 17,  13, 413]), array([ 17,  12, 413]), array([ 17,  11, 413]), array([ 17,  10, 413]), array([ 17,   9, 413]), array([ 17,   8, 413]), array([ 17,   7, 413]), array([ 17,   6, 413]), array([ 17,   5, 413]), array([ 17,   4, 405]), array([ 17,   3, 384]), array([ 17,   2, 326]), array([ 17,   1, 194]), array([17,  0,  0]), array([ 16,  15, 413]), array([ 16,  14, 413]), array([ 16,  13, 413]), array([ 16,  12, 413]), array([ 16,  11, 413]), array([ 16,  10, 413]), array([ 16,   9, 413]), array([ 16,   8, 413]), array([ 16,   7, 413]), array([ 16,   6, 413]), array([ 16,   5, 413]), array([ 16,   4, 405]), array([ 16,   3, 384]), array([ 16,   2, 326]), array([ 16,   1, 194]), array([16,  0,  0]), array([ 15,  15, 413]), array([ 15,  14, 413]), array([ 15,  13, 413]), array([ 15,  12, 413]), array([ 15,  11, 413]), array([ 15,  10, 413]), array([ 15,   9, 413]), array([ 15,   8, 413]), array([ 15,   7, 413]), array([ 15,   6, 413]), array([ 15,   5, 413]), array([ 15,   4, 405]), array([ 15,   3, 384]), array([ 15,   2, 326]), array([ 15,   1, 194]), array([15,  0,  0]), array([ 14,  15, 413]), array([ 14,  14, 413]), array([ 14,  13, 413]), array([ 14,  12, 413]), array([ 14,  11, 413]), array([ 14,  10, 413]), array([ 14,   9, 413]), array([ 14,   8, 413]), array([ 14,   7, 413]), array([ 14,   6, 413]), array([ 14,   5, 413]), array([ 14,   4, 405]), array([ 14,   3, 384]), array([ 14,   2, 326]), array([ 14,   1, 194]), array([14,  0,  0]), array([ 13,  15, 413]), array([ 13,  14, 413]), array([ 13,  13, 413]), array([ 13,  12, 413]), array([ 13,  11, 413]), array([ 13,  10, 413]), array([ 13,   9, 413]), array([ 13,   8, 413]), array([ 13,   7, 413]), array([ 13,   6, 413]), array([ 13,   5, 413]), array([ 13,   4, 405]), array([ 13,   3, 384]), array([ 13,   2, 326]), array([ 13,   1, 194]), array([13,  0,  0]), array([ 12,  15, 413]), array([ 12,  14, 413]), array([ 12,  13, 413]), array([ 12,  12, 413]), array([ 12,  11, 413]), array([ 12,  10, 413]), array([ 12,   9, 413]), array([ 12,   8, 413]), array([ 12,   7, 413]), array([ 12,   6, 413]), array([ 12,   5, 413]), array([ 12,   4, 405]), array([ 12,   3, 384]), array([ 12,   2, 326]), array([ 12,   1, 194]), array([12,  0,  0]), array([ 11,  15, 413]), array([ 11,  14, 413]), array([ 11,  13, 413]), array([ 11,  12, 413]), array([ 11,  11, 413]), array([ 11,  10, 413]), array([ 11,   9, 413]), array([ 11,   8, 413]), array([ 11,   7, 413]), array([ 11,   6, 413]), array([ 11,   5, 413]), array([ 11,   4, 404]), array([ 11,   3, 379]), array([ 11,   2, 325]), array([ 11,   1, 180]), array([11,  0,  0]), array([ 10,  15, 413]), array([ 10,  14, 413]), array([ 10,  13, 413]), array([ 10,  12, 413]), array([ 10,  11, 413]), array([ 10,  10, 413]), array([ 10,   9, 413]), array([ 10,   8, 413]), array([ 10,   7, 413]), array([ 10,   6, 413]), array([ 10,   5, 413]), array([ 10,   4, 404]), array([ 10,   3, 379]), array([ 10,   2, 325]), array([ 10,   1, 180]), array([10,  0,  0]), array([  9,  15, 413]), array([  9,  14, 413]), array([  9,  13, 413]), array([  9,  12, 413]), array([  9,  11, 413]), array([  9,  10, 413]), array([  9,   9, 413]), array([  9,   8, 413]), array([  9,   7, 413]), array([  9,   6, 413]), array([  9,   5, 410]), array([  9,   4, 400]), array([  9,   3, 379]), array([  9,   2, 325]), array([  9,   1, 180]), array([9, 0, 0]), array([  8,  15, 413]), array([  8,  14, 413]), array([  8,  13, 413]), array([  8,  12, 413]), array([  8,  11, 413]), array([  8,  10, 413]), array([  8,   9, 413]), array([  8,   8, 413]), array([  8,   7, 413]), array([  8,   6, 413]), array([  8,   5, 410]), array([  8,   4, 397]), array([  8,   3, 369]), array([  8,   2, 302]), array([  8,   1, 177]), array([8, 0, 0])]
points = array(points)
print(points)

[[ 22  15 413]
 [ 22  14 413]
 [ 22  13 413]
 [ 22  12 413]
 [ 22  11 413]
 [ 22  10 413]
 [ 22   9 413]
 [ 22   8 413]
 [ 22   7 413]
 [ 22   6 413]
 [ 22   5 413]
 [ 22   4 405]
 [ 22   3 384]
 [ 22   2 326]
 [ 22   1 194]
 [ 22   0   0]
 [ 21  15 413]
 [ 21  14 413]
 [ 21  13 413]
 [ 21  12 413]
 [ 21  11 413]
 [ 21  10 413]
 [ 21   9 413]
 [ 21   8 413]
 [ 21   7 413]
 [ 21   6 413]
 [ 21   5 413]
 [ 21   4 405]
 [ 21   3 384]
 [ 21   2 326]
 [ 21   1 194]
 [ 21   0   0]
 [ 20  15 413]
 [ 20  14 413]
 [ 20  13 413]
 [ 20  12 413]
 [ 20  11 413]
 [ 20  10 413]
 [ 20   9 413]
 [ 20   8 413]
 [ 20   7 413]
 [ 20   6 413]
 [ 20   5 413]
 [ 20   4 405]
 [ 20   3 384]
 [ 20   2 326]
 [ 20   1 194]
 [ 20   0   0]
 [ 19  15 413]
 [ 19  14 413]
 [ 19  13 413]
 [ 19  12 413]
 [ 19  11 413]
 [ 19  10 413]
 [ 19   9 413]
 [ 19   8 413]
 [ 19   7 413]
 [ 19   6 413]
 [ 19   5 413]
 [ 19   4 405]
 [ 19   3 384]
 [ 19   2 326]
 [ 19   1 194]
 [ 19   0   0]
 [ 18  15 413]
 [ 18  14 413]
 [ 18  13 

In [29]:
Z = np.array(points[:,2]).reshape(15 ,n_jobs+1)

X = np.linspace(horizon,8, 15)
Y = np.linspace(n_jobs,0,n_jobs +1)
X, Y = np.meshgrid(X, Y)


fig = go.Figure(data=[go.Surface(x=X, y=Y, z=Z.T)])
fig.update_layout(scene=dict(xaxis_title="Days", yaxis_title="Jobs", zaxis_title="Benefit"))
fig.show()